In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
from utils import *

In [3]:
class Module:
    """
    Module is a super class. It could be a single layer, or a multilayer perceptron.
    """
    
    def __init__(self):
        self.train = True
        return
    
    def forward(self, _input):
        """
        h = f(z); z is the input, and h is the output.
        
        Inputs:
        _input: z
        
        Returns:
        output h
        """
        pass
    
    def backward(self, _input, _gradOutput):
        """
        Compute:
        gradient w.r.t. _input
        gradient w.r.t. trainable parameters
        
        Inputs:
        _input: z
        _gradOutput: dL/dh
        
        Returns:
        gradInput: dL/dz
        """
        pass
        
    def parameters(self):
        """
        Return the value of trainable parameters and its corresponding gradient (Used for grandient descent)
        
        Returns:
        params, gradParams
        """
        pass
    
    def training(self):
        """
        Turn the module into training mode. (Only useful for Dropout layer)
        Ignore it if you are not using Dropout.
        """
        self.train = True
        
    def evaluate(self):
        """
        Turn the module into evaluate mode. (Only useful for Dropout layer)
        Ignore it if you are not using Dropout.
        """
        self.train = False

In [4]:
class Sequential(Module):
    def __init__(self):
        Module.__init__(self)
        self.layers = []
    
    def add(self, layer):
        self.layers.append(layer)
    
    def size(self):
        return len(self.layers)
    
    def forward(self, _input):
        self._inputs = [_input]
        for layer in self.layers: self._inputs.append(layer.forward(self._inputs[-1]))
        return self._inputs[-1]
    
    def backward(self, _input, _gradOutput):
        if self._inputs is None:
            print('had to do this')
            self.forward(_input)
        
        self._gradInputs = [None for _ in range(self.size() + 1)]
        self._gradInputs[self.size()] = _gradOutput

        for i in range(self.size()-1, -1, -1):
            self._gradInputs[i] = self.layers[i].backward(self._inputs[i], self._gradInputs[i+1])
        
        return self._gradInputs[0]
    
    def parameters(self):
        params = []
        gradParams = []
        for m in self.layers:
            _p, _g = m.parameters()
            if _p is not None:
                params.append(_p)
                gradParams.append(_g)
        return params, gradParams

    def training(self):
        Module.training(self)
        for m in self.layers:
            m.training()
    
    def evaluate(self):
        Module.evaluate(self)
        for m in self.layers:
            m.evaluate()

In [5]:
class FullyConnected(Module):
    def __init__(self, inputSize, outputSize):
        Module.__init__(self)
        stdv = 1. / np.sqrt(inputSize)
        self.weight = np.random.uniform(-stdv, stdv, (inputSize, outputSize))
        self.gradWeight = np.ndarray((inputSize, outputSize))
        self.bias = np.random.uniform(-stdv, stdv, outputSize)
        self.gradBias = np.ndarray(outputSize)
        
    def forward(self, _input):
        """
        _input: N x inputSize
        """
        return  _input @ self.weight + self.bias
    
    def backward(self, _input, _gradOutput):
        """
        _input: N x inputSize
        _gradOutput: N x outputSize
        """
        self.gradWeight = _input.T @ _gradOutput
        self.gradBias = np.sum(_gradOutput, axis=0)
        return _gradOutput @ self.weight.T
        
    def parameters(self):
        return [self.weight, self.bias], [self.gradWeight, self.gradBias]

In [8]:
class ReLU(Module):
    """
    ReLU activation, not trainable.
    """
    def __init__(self):
        Module.__init__(self)
    
    def forward(self, _input):
        """
        _input: N x d
        """
        return _input * (_input > 0)
    
    def backward(self, _input, _gradOutput):
        """
        _input: N x d
        _gradOutput: N x d
        """
        return _gradOutput * (_input > 0)
        
    def parameters(self):
        """
        No trainable parameters, return None
        """
        return None, None

In [29]:
'''
class Logistic(Module):
    """
    Logistic activation, not trainable.
    """
    def __init__(self):
        Module.__init__(self)
    
    def forward(self, _input):
        pass
    
    def backward(self, _input, _gradOutput):
        pass
        
    def parameters(self):
        return None, None
'''

class Sigmoid(Module):
    def __init__(self):
        Module.__init__(self)
        
    @staticmethod
    def sigmoid(x):
        return 1 / (1 + np.exp(-x))
        
    def forward(self, _input):
        return self.sigmoid(_input)
    
    def backward(self, _input, _gradOutput):
        return _gradOutput * self.sigmoid(_input) * (1 - self.sigmoid(_input))
    
    def parameters(self):
        return None, None

In [ ]:
'''
class Dropout(Module):
    """
    A dropout layer
    """
    def __init__(self, p = 0.5):
        Module.__init__(self)
        self.p = p
        
    def forward(self, _input):
        pass
    
    def backward(self, _input, _gradOutput):
        pass
    
    def parameters(self):
        return None, None
'''

In [9]:
from scipy.special import logsumexp

class SoftMaxLoss(object):
    def __init__(self):
        return
    
    @staticmethod
    def logits(_input):
        return _input - logsumexp(_input, axis=1)[:, np.newaxis]
        
    def forward(self, _input, _label):
        """
        _input: N x C
        _labels: N x C, one-hot
        """
        xent = np.sum(self.logits(_input) * _label)
        return -xent / _input.shape[0]
    
    def backward(self, _input, _label):
        probs = np.exp(self.logits(_input))
        return (probs - _label) / _input.shape[0]

In [42]:
def test_sm():
    crit = SoftMaxLoss()
    gt = np.zeros((3, 10))
    gt[np.arange(3), np.array([1,2,3])] = 1
    x = np.random.random((3,10))
    
    gradInput = crit.backward(x, gt)
    gradInput_num = numeric_gradient(lambda x: crit.forward(x, gt), x, 1, 1e-6)
    
    print(gradInput, '\n')
    print(gradInput_num, '\n')
    print(relative_error(gradInput, gradInput_num, 1e-8))
    
test_sm()

[[ 0.02254328 -0.31089954  0.02081726  0.02107968  0.04382796  0.02676907
   0.04459196  0.04099092  0.04747656  0.04280285]
 [ 0.05323232  0.03528053 -0.28669873  0.02875008  0.02587762  0.0329954
   0.02502313  0.0352213   0.02558782  0.02473053]
 [ 0.02522514  0.03811736  0.03078141 -0.29795639  0.03458867  0.04492276
   0.02989044  0.03091896  0.04091967  0.02259198]] 

[[ 0.02254328 -0.31089954  0.02081726  0.02107968  0.04382796  0.02676907
   0.04459196  0.04099092  0.04747656  0.04280285]
 [ 0.05323232  0.03528053 -0.28669873  0.02875008  0.02587762  0.0329954
   0.02502313  0.0352213   0.02558782  0.02473053]
 [ 0.02522514  0.03811736  0.03078141 -0.29795639  0.03458867  0.04492276
   0.02989044  0.03091896  0.04091967  0.02259198]] 

4.204230978652034e-09


In [30]:
def test_module(model):
    model.evaluate()

    crit = TestCriterion()
    gt = np.random.random((3,10))
    x = np.random.random((3,10))

    gradInput = model.backward(x, crit.backward(model.forward(x), gt))
    gradInput_num = numeric_gradient(lambda x: crit.forward(model.forward(x), gt), x, 1, 1e-6)
    print(type(model))
    print(relative_error(gradInput, gradInput_num, 1e-8))

# Test fully connected
model = FullyConnected(10, 10)
test_module(model)

# Test ReLU
model = ReLU()
test_module(model)

# Test Sigmoid 
model = Sigmoid()
test_module(model)

# Test Sequential
model = Sequential()
model.add(FullyConnected(10, 10))
model.add(ReLU())
test_module(model)

<class '__main__.FullyConnected'>
4.332481191418781e-09
<class '__main__.ReLU'>
2.631778962118705e-10
<class '__main__.Sigmoid'>
3.918167996349233e-09
<class '__main__.Sequential'>
2.0692114196892912e-08


In [ ]:
# Test gradient descent, the loss should be lower and lower
trainX = np.random.random((10,5))

model = Sequential()
model.add(FullyConnected(5, 3))
model.add(ReLU())
model.add(FullyConnected(3, 1))

crit = TestCriterion()

it = 0
state = None
while it <= 1000:
    output = model.forward(trainX)
    loss = crit.forward(output, None)
    if it % 100 == 0: print(loss)
    
    doutput = crit.backward(output, None)
    model.backward(trainX, doutput)
    
    params, gradParams = model.parameters()
    sgd(params, gradParams, 0.01, 0.8)
    it += 1

Now we start to work on real data.

In [14]:
import MNIST_utils
data_fn = "CLEAN_MNIST_SUBSETS.h5"

# We only consider large set this time
print("Load large trainset.")
Xlarge, Ylarge = MNIST_utils.load_data(data_fn, "large_train")
print(Xlarge.shape)
print(Ylarge.shape)

print("Load valset.")
Xval, Yval = MNIST_utils.load_data(data_fn, "val")
print(Xval.shape)
print(Yval.shape)

Load large trainset.
(7000, 576)
(7000, 10)
Load valset.
(2000, 576)
(2000, 10)


In [37]:
def predict(X, model):
    """
    Evaluate the soft predictions of the model.
    Input:
    X : N x d array (no unit terms)
    model : a multi-layer perceptron
    Output:
    yhat : N x C array
        yhat[n][:] contains the score over C classes for X[n][:]
    """
    return model.forward(X)

def error_rate(X, Y, model):
    """
    Compute error rate (between 0 and 1) for the model
    """
    model.evaluate()
    res = 1 - (model.forward(X).argmax(-1) == Y.argmax(-1)).mean()
    model.training()
    return res

from copy import deepcopy

def runTrainVal(X, Y, model, Xval, Yval, trainopt):
    """
    Run the train + evaluation on a given train/val partition
    trainopt: various (hyper)parameters of the training procedure
    During training, choose the model with the lowest validation error. (early stopping)
    """
    
    eta = trainopt['eta']
    
    N = X.shape[0] # number of data points in X
    
    # Save the model with lowest validation error
    minValError = np.inf
    saved_model = None
    
    shuffled_idx = np.random.permutation(N)
    start_idx = 0
    last_val = None
    for iteration in range(trainopt['maxiter']):
        if iteration % int(trainopt['eta_frac'] * trainopt['maxiter']) == 0:
            eta *= trainopt['etadrop']
            
        # form the next mini-batch
        stop_idx = min(start_idx + trainopt['batch_size'], N)
        batch_idx = range(N)[int(start_idx):int(stop_idx)]
        bX = X[shuffled_idx[batch_idx],:]
        bY = Y[shuffled_idx[batch_idx],:]

        score = model.forward(bX)
        loss = crit.forward(score, bY)
        dscore = crit.backward(score, bY)
        model.backward(bX, dscore)
        
        # update
        params, gradParams = model.parameters()
        sgd(params, gradParams, eta, weight_decay = trainopt['lambda'])    
        start_idx = stop_idx % N
        
        if (iteration % trainopt['display_iter']) == 0:
            # compute train and val error; multiply by 100 for readability (make it percentage points)
            trainError = 100 * error_rate(X, Y, model)
            valError = 100 * error_rate(Xval, Yval, model)
            print('{:8} batch loss: {:.3f} train error: {:.3f} val error: {:.3f}'.format(iteration, loss, trainError, valError))
            
            # early stopping criterion
            # if last_val is not None and valError > last_val * (1 - 1e-3): break
            # last_val = valError
            
            if valError < minValError:
                saved_model = deepcopy(model)
                minValError = valError
        
    return saved_model, minValError, trainError

In [31]:
activation = {'ReLU': ReLU, 'Sigmoid': Sigmoid}

def build_model(input_size, hidden_size, output_size, activation_func='ReLU', dropout=0):
    """
    Build the model:
    input_size: the dimension of input data
    hidden_size: the dimension of hidden vector
    output_size: the output size of final layer.
    activation_func: ReLU, Logistic, Tanh, etc. (Need to be implemented by yourself)
    dropout: the dropout rate: if dropout == 0, this is equivalent to no dropout
    """
    model = Sequential()
    model.add(FullyConnected(input_size, hidden_size))
    model.add(activation[activation_func]())
    model.add(FullyConnected(hidden_size, output_size))
    return model

def build_model_custom(input_size, *hidden_sizes, output_size=10, activation_func='ReLU', dropout=0):
    model = Sequential()
    last_size = input_size
    for hidden_size in hidden_sizes:
        model.add(FullyConnected(last_size, hidden_size))
        model.add(activation[activation_func]())
        last_size = hidden_size
    model.add(FullyConnected(last_size, output_size))
    return model

In [48]:
# -- training options
trainopt = {
    'eta': .1,   # initial learning rate
    'maxiter': 5000,   # max number of iterations (updates) of SGD
    'display_iter': 500,  # display batch loss every display_iter updates
    'batch_size': 100,  
    'etadrop': .75, # when dropping eta, multiply it by this number (e.g., .5 means halve it)
    'eta_frac': .25,
    'lambda': 0.01
}

In [ ]:
NFEATURES = Xlarge.shape[1]
trained_models = dict()

# set the (initial?) set of lambda values to explore
lambdas = np.array([0, 0.001, 0.01, 0.1])
hidden_sizes = np.array([10])
    
for lambda_ in lambdas:
    for hidden_size_ in hidden_sizes:
        trainopt['lambda'] = lambda_
        model = build_model(NFEATURES, hidden_size_, 10, dropout=0)
        crit = SoftMaxLoss()
        trained_model, valErr, trainErr = runTrainVal(Xlarge, Ylarge, model, Xval, Yval, trainopt)
        trained_models[(lambda_, hidden_size_)] = {'model': trained_model, "val_err": valErr, "train_err": trainErr }
        print('train set model: -> lambda= %.4f, train error: %.2f, val error: %.2f' % (lambda_, trainErr, valErr))
    
best_trained_lambda = 0.
best_trained_model = None
best_trained_val_err = 100.
for (lambda_, hidden_size_), results in trained_models.items():
    print('lambda= %.4f, hidden size: %5d, val error: %.2f' %(lambda_, hidden_size_, results['val_err']))
    if results['val_err'] < best_trained_val_err:
        best_trained_val_err = results['val_err']
        best_trained_model = results['model']
        best_trained_lambda = lambda_

print("Best train model val err:", best_trained_val_err)
print("Best train model lambda:", best_trained_lambda)

In [50]:
# custom model architectures
custom_model = build_model_custom(NFEATURES, 400, 400, output_size=10, activation_func='ReLU')
crit = SoftMaxLoss()
trained_model, valErr, trainErr = runTrainVal(Xlarge, Ylarge, custom_model, Xval, Yval, trainopt)
print(f'train error: {trainErr:.2f}, val error: {valErr:.2f}')

       0 batch loss: 2.303 train error: 84.714 val error: 85.300
     500 batch loss: 0.311 train error: 7.357 val error: 7.350
    1000 batch loss: 0.204 train error: 5.257 val error: 6.650
    1500 batch loss: 0.129 train error: 3.271 val error: 5.500
    2000 batch loss: 0.094 train error: 1.957 val error: 5.100
    2500 batch loss: 0.067 train error: 1.214 val error: 4.700
    3000 batch loss: 0.029 train error: 0.829 val error: 4.850
    3500 batch loss: 0.034 train error: 0.614 val error: 4.650
    4000 batch loss: 0.038 train error: 0.357 val error: 4.550
    4500 batch loss: 0.024 train error: 0.229 val error: 4.450
train error: 0.23, val error: 4.45


In [52]:
model = Sequential()
model.add(FullyConnected(NFEATURES, 200))
model.add(ReLU())
model.add(FullyConnected(200, 80))
model.add(Sigmoid())
model.add(FullyConnected(80, 10))
crit = SoftMaxLoss()

trained_model, valErr, trainErr = runTrainVal(Xlarge, Ylarge, model, Xval, Yval, trainopt)
print(f'train error: {trainErr:.2f}, val error: {valErr:.2f}')

       0 batch loss: 2.335 train error: 89.429 val error: 89.700
     500 batch loss: 0.784 train error: 21.214 val error: 18.950
    1000 batch loss: 0.499 train error: 11.386 val error: 10.150
    1500 batch loss: 0.250 train error: 8.629 val error: 8.400
    2000 batch loss: 0.293 train error: 7.486 val error: 7.800
    2500 batch loss: 0.128 train error: 6.600 val error: 7.250
    3000 batch loss: 0.176 train error: 6.271 val error: 7.200
    3500 batch loss: 0.229 train error: 5.700 val error: 6.550
    4000 batch loss: 0.246 train error: 5.286 val error: 6.100
    4500 batch loss: 0.251 train error: 5.000 val error: 5.800
train error: 5.00, val error: 5.80


In [53]:
kaggleX = MNIST_utils.load_data(data_fn, 'kaggle')
kaggleYhat = predict(kaggleX, trained_model).argmax(-1)
save_submission('submission-mnist.csv', kaggleYhat)

Saved: submission-mnist.csv


In [85]:
kaggleX = MNIST_utils.load_data(data_fn, 'kaggle')
kaggleYhat = predict(kaggleX, best_trained_model).argmax(-1)
save_submission('submission-mnist.csv', kaggleYhat)

Saved: submission-mnist.csv
